In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv1D, MaxPooling1D, LSTM, Dense, Dropout, BatchNormalization
)
from tensorflow.keras.optimizers import Adam

# ==================== HYBRID FOCAL LOSS ====================
def hybrid_focal_loss(alpha=0.25, gamma=2.0, bce_weight=0.5):
    """
    Hybrid = BCE * bce_weight + FocalLoss * (1 - bce_weight)
    """
    def loss(y_true, y_pred):
        # --- Binary Cross Entropy ---
        bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)

        # --- Focal Loss ---
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        focal = -alpha * (1 - y_pred) ** gamma * y_true * K.log(y_pred) \
                - (1 - alpha) * y_pred ** gamma * (1 - y_true) * K.log(1 - y_pred)

        focal = K.mean(focal, axis=-1)

        # --- Combine Both ---
        return bce_weight * bce + (1 - bce_weight) * focal

    return loss

# ==================== CNN + LSTM MODEL ====================
def build_cnn_lstm(input_length):
    model = Sequential([
        # --- CNN Layers ---
        Conv1D(32, kernel_size=7, activation='relu', input_shape=(input_length, 1)),
        BatchNormalization(),
        MaxPooling1D(2),
        Dropout(0.2),

        Conv1D(64, kernel_size=5, activation='relu'),
        BatchNormalization(),
        MaxPooling1D(2),
        Dropout(0.3),

        Conv1D(128, kernel_size=3, activation='relu'),
        BatchNormalization(),
        MaxPooling1D(2),
        Dropout(0.4),
        
        # --- LSTM ---
        LSTM(64, return_sequences=False),

        # --- Dense Layers ---
        Dense(64, activation='relu'),
        Dropout(0.4),

        Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=Adam(1e-4),
        loss=hybrid_focal_loss(alpha=0.25, gamma=2.0, bce_weight=0.5),
        metrics=['accuracy']
    )
    return model

# ==================== DATA AUGMENTATION ====================
def augment_signal(signal):
    # 1) Very Light Noise
    noise = np.random.normal(0, 0.005, signal.shape)
    signal_noisy = signal + noise

    # 2) Small Time Shift
    shift = np.random.randint(-5, 5)
    signal_shifted = np.roll(signal_noisy, shift)

    # 3) Gentle Scaling
    scale = np.random.uniform(0.97, 1.03)
    signal_scaled = signal_shifted * scale

    return signal_scaled

def augment_batch(X, y):
    X_aug = []
    y_aug = []

    for i in range(len(X)):
        X_aug.append(X[i])
        y_aug.append(y[i])

        # Generate **1 weakly augmented version**
        X_aug.append(augment_signal(X[i]))
        y_aug.append(y[i])

    return np.array(X_aug), np.array(y_aug)

# ==================== LOAD PRE-SPLIT FOLD DATA ====================
print("="*70)
print("LOADING PRE-SPLIT 5-FOLD DATA (No Data Leakage)")
print("="*70)

preprocessed_dir = r"preprocessed_5fold"

# Check if fold data exists
if not os.path.exists(preprocessed_dir):
    print(f"❌ Error: {preprocessed_dir} not found!")
    print("Please run the preprocessing script first to create the folds.")
    exit(1)

# Load fold indices for reference
fold_file_indices = np.load(os.path.join(preprocessed_dir, "fold_file_indices.npy"), allow_pickle=True)
print(f"✅ Found {len(fold_file_indices)} folds")

# ==================== TRAINING ====================
os.makedirs("results(Interictal_VS_Normal_VS_Ictal)", exist_ok=True)

# Save the fold indices for reproducibility
np.save("results(Interictal_VS_Normal_VS_Ictal)/fold_indices.npy", fold_file_indices, allow_pickle=True)

acc_per_fold = []
conf_matrices = []

for fold_no in range(1, len(fold_file_indices) + 1):
    print("\n" + "="*70)
    print(f"FOLD {fold_no}")
    print("="*70)
    
    # Load pre-split fold data
    X_train_val = np.load(os.path.join(preprocessed_dir, f"fold{fold_no}_train_X.npy"))
    y_train_val = np.load(os.path.join(preprocessed_dir, f"fold{fold_no}_train_y.npy"))
    X_test = np.load(os.path.join(preprocessed_dir, f"fold{fold_no}_test_X.npy"))
    y_test = np.load(os.path.join(preprocessed_dir, f"fold{fold_no}_test_y.npy"))
    
    # Convert to binary: ICTAL = 0, ALL OTHERS = 1
    y_train_val_encoded = np.where(y_train_val == 'ICTAL', 0, 1)
    y_test_encoded = np.where(y_test == 'ICTAL', 0, 1)
    
    # Split train into train/val
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_val, y_train_val_encoded, test_size=0.1765, stratify=y_train_val_encoded, random_state=42
    )
    
    # Reshape for CNN
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
    
    print(f"\nFold {fold_no} data shapes:")
    print(f"  Train (before aug): {X_train.shape}")
    print(f"  Val:                {X_val.shape}")
    print(f"  Test:               {X_test.shape}")
    
    print(f"\nClass distribution:")
    print(f"  Train: {np.unique(y_train, return_counts=True)}")
    print(f"  Val:   {np.unique(y_val, return_counts=True)}")
    print(f"  Test:  {np.unique(y_test_encoded, return_counts=True)}")
    
    # ==================== APPLY DATA AUGMENTATION ====================
    print(f"\n🔄 Applying data augmentation...")
    X_train, y_train = augment_batch(X_train, y_train)
    print(f"  After augmentation: {X_train.shape}")
    
    # ==================== BUILD AND TRAIN MODEL ====================
    print(f"\n🏗️  Building model...")
    model = build_cnn_lstm(X_train.shape[1])
    
    if fold_no == 1:
        model.summary()
    
    # Handle class imbalance
    cw = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
    class_weights = {0: cw[0], 1: cw[1]}
    print(f"  Class weights: {class_weights}")
    
    # Train model
    print(f"\n🚀 Training Fold {fold_no}...")
    history = model.fit(
        X_train, y_train,
        epochs=40,
        batch_size=32,
        validation_data=(X_val, y_val),
        class_weight=class_weights,
        verbose=1
    )
    
    # ==================== EVALUATE ====================
    test_loss, test_acc = model.evaluate(X_test, y_test_encoded, verbose=0)
    acc_per_fold.append(test_acc)
    
    print(f"\n✅ Fold {fold_no} Results:")
    print(f"  Test Accuracy: {test_acc:.4f}")
    print(f"  Test Loss:     {test_loss:.4f}")
    
    # Save weights
    weight_path = f"results(Interictal_VS_Normal_VS_Ictal)/cnn_lstm_fold{fold_no}.weights.h5"
    model.save_weights(weight_path)
    print(f"  💾 Weights saved: {weight_path}")
    
    # ==================== PREDICTIONS & CONFUSION MATRIX ====================
    y_pred = (model.predict(X_test, verbose=0) > 0.5).astype("int32").flatten()
    cm = confusion_matrix(y_test_encoded, y_pred)
    conf_matrices.append(cm)
    
    # Print classification report
    print(f"\nFold {fold_no} Classification Report:")
    print(classification_report(
        y_test_encoded, y_pred,
        target_names=['ICTAL', 'Others (Normal+Interictal)'],
        digits=4
    ))
    
    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(
        cm, annot=True, fmt='d', cmap='Blues',
        xticklabels=['ICTAL', 'Others'],
        yticklabels=['ICTAL', 'Others'],
        annot_kws={'size': 14}
    )
    plt.title(f"Fold {fold_no} Confusion Matrix (No Data Leakage)\nAccuracy: {test_acc:.4f}", fontsize=14)
    plt.xlabel("Predicted", fontsize=12)
    plt.ylabel("True", fontsize=12)
    plt.tight_layout()
    plt.savefig(f"results(Interictal_VS_Normal_VS_Ictal)/cnn_lstm_conf_fold{fold_no}.png", dpi=300)
    plt.close()
    
    # Plot training history
    plt.figure(figsize=(14, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy', linewidth=2)
    plt.plot(history.history['val_accuracy'], label='Val Accuracy', linewidth=2)
    plt.axhline(y=test_acc, color='r', linestyle='--', label=f'Test Acc: {test_acc:.4f}', linewidth=2)
    plt.title(f'Fold {fold_no} - Model Accuracy', fontsize=13)
    plt.xlabel('Epoch', fontsize=11)
    plt.ylabel('Accuracy', fontsize=11)
    plt.legend(fontsize=10)
    plt.grid(alpha=0.3)
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss', linewidth=2)
    plt.plot(history.history['val_loss'], label='Val Loss', linewidth=2)
    plt.title(f'Fold {fold_no} - Model Loss', fontsize=13)
    plt.xlabel('Epoch', fontsize=11)
    plt.ylabel('Loss', fontsize=11)
    plt.legend(fontsize=10)
    plt.grid(alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(f"results(Interictal_VS_Normal_VS_Ictal)/cnn_lstm_history_fold{fold_no}.png", dpi=300)
    plt.close()

# ==================== OVERALL RESULTS ====================
print("\n" + "="*70)
print("CROSS-VALIDATION SUMMARY (No Data Leakage)")
print("="*70)

print(f"\n📊 Mean Test Accuracy: {np.mean(acc_per_fold):.4f} (±{np.std(acc_per_fold):.4f})")
print(f"\nAccuracy per fold:")
for i, acc in enumerate(acc_per_fold, start=1):
    print(f"  Fold {i}: {acc:.4f}")

# ==================== OVERALL CONFUSION MATRIX ====================
total_cm = np.sum(conf_matrices, axis=0)

plt.figure(figsize=(8, 6))
sns.heatmap(
    total_cm, annot=True, fmt='d', cmap='Greens',
    xticklabels=['ICTAL', 'Others'],
    yticklabels=['ICTAL', 'Others'],
    annot_kws={'size': 14}
)
plt.title("Overall Confusion Matrix (All Folds - No Data Leakage)", fontsize=14)
plt.xlabel("Predicted", fontsize=12)
plt.ylabel("True", fontsize=12)
plt.tight_layout()
plt.savefig("results(Interictal_VS_Normal_VS_Ictal)/cnn_lstm_conf_overall.png", dpi=300)
plt.show()

# Calculate overall metrics
tn, fp, fn, tp = total_cm.ravel()

overall_accuracy = (tp + tn) / np.sum(total_cm)
overall_precision = tp / (tp + fp) if (tp + fp) > 0 else 0
overall_recall = tp / (tp + fn) if (tp + fn) > 0 else 0
overall_f1 = 2 * overall_precision * overall_recall / (overall_precision + recall) if (overall_precision + overall_recall) > 0 else 0

print("\n📊 Overall Metrics (Across All Folds):")
print(f"  Accuracy:  {overall_accuracy:.4f}")
print(f"  Precision: {overall_precision:.4f}")
print(f"  Recall:    {overall_recall:.4f}")
print(f"  F1-score:  {overall_f1:.4f}")

print("\n" + "="*70)
print("✅ CNN+LSTM 5-FOLD CROSS-VALIDATION COMPLETED!")
print("="*70)
print("\nKey features:")
print("  ✅ Files split into 5 folds BEFORE segmentation")
print("  ✅ No file overlap between train/test in each fold")
print("  ✅ Data augmentation applied only to training set")
print("  ✅ Validation set created from training fold")
print("  ✅ Same model architecture and loss as original")
print("  ✅ Same logic, just fixed data leakage issue")

LOADING PRE-SPLIT 5-FOLD DATA (No Data Leakage)
✅ Found 5 folds

FOLD 1

Fold 1 data shapes:
  Train (before aug): (7246, 347, 1)
  Val:                (1554, 347, 1)
  Test:               (2200, 347, 1)

Class distribution:
  Train: (array([0, 1]), array([1449, 5797]))
  Val:   (array([0, 1]), array([ 311, 1243]))
  Test:  (array([0, 1]), array([ 440, 1760]))

🔄 Applying data augmentation...
  After augmentation: (14492, 347, 1)

🏗️  Building model...


c:\Users\Mostafa\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_3 (Conv1D)               │ (None, 341, 32)        │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 341, 32)        │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (None, 170, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 170, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, 166, 64)        │        10,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 166, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_4 (MaxPooling1D)  │ (None, 83, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 83, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ (None, 81, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 81, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_5 (MaxPooling1D)  │ (None, 40, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 40, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 89,793 (350.75 KB)

...

  Class weights: {0: np.float64(2.500345065562457), 1: np.float64(0.6249784371226497)}

🚀 Training Fold 1...
Epoch 1/40
453/453 ━━━━━━━━━━━━━━━━━━━━ 30s 54ms/step - accuracy: 0.5219 - loss: 0.3690 - val_accuracy: 0.8546 - val_loss: 0.3071
Epoch 2/40
453/453 ━━━━━━━━━━━━━━━━━━━━ 23s 51ms/step - accuracy: 0.8367 - loss: 0.2084 - val_accuracy: 0.9453 - val_loss: 0.1113
Epoch 3/40
453/453 ━━━━━━━━━━━━━━━━━━━━ 22s 49ms/step - accuracy: 0.9436 - loss: 0.1035 - val_accuracy: 0.9530 - val_loss: 0.0925
Epoch 4/40
453/453 ━━━━━━━━━━━━━━━━━━━━ 23s 50ms/step - accuracy: 0.9568 - loss: 0.0838 - val_accuracy: 0.9653 - val_loss: 0.0742
Epoch 5/40
453/453 ━━━━━━━━━━━━━━━━━━━━ 22s 49ms/step - accuracy: 0.9603 - loss: 0.0753 - val_accuracy: 0.9691 - val_loss: 0.0676
Epoch 6/40
453/453 ━━━━━━━━━━━━━━━━━━━━ 22s 49ms/step - accuracy: 0.9656 - loss: 0.0625 - val_accuracy: 0.9717 - val_loss: 0.0637
Epoch 7/40
453/453 ━━━━━━━━━━━━━━━━━━━━ 22s 49ms/step - accuracy: 0.9637 - loss: 0.0609 - val_accuracy: 0.9781 

c:\Users\Mostafa\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


453/453 ━━━━━━━━━━━━━━━━━━━━ 28s 50ms/step - accuracy: 0.4839 - loss: 0.3749 - val_accuracy: 0.7793 - val_loss: 0.3197
Epoch 2/40
453/453 ━━━━━━━━━━━━━━━━━━━━ 22s 49ms/step - accuracy: 0.8296 - loss: 0.2106 - val_accuracy: 0.9601 - val_loss: 0.0997
Epoch 3/40
453/453 ━━━━━━━━━━━━━━━━━━━━ 22s 49ms/step - accuracy: 0.9498 - loss: 0.0939 - val_accuracy: 0.9614 - val_loss: 0.0834
Epoch 4/40
453/453 ━━━━━━━━━━━━━━━━━━━━ 22s 49ms/step - accuracy: 0.9585 - loss: 0.0760 - val_accuracy: 0.9736 - val_loss: 0.0647
Epoch 5/40
453/453 ━━━━━━━━━━━━━━━━━━━━ 22s 49ms/step - accuracy: 0.9661 - loss: 0.0605 - val_accuracy: 0.9801 - val_loss: 0.0549
Epoch 6/40
453/453 ━━━━━━━━━━━━━━━━━━━━ 22s 49ms/step - accuracy: 0.9709 - loss: 0.0541 - val_accuracy: 0.9807 - val_loss: 0.0467
Epoch 7/40
453/453 ━━━━━━━━━━━━━━━━━━━━ 22s 49ms/step - accuracy: 0.9744 - loss: 0.0463 - val_accuracy: 0.9807 - val_loss: 0.0415
Epoch 8/40
453/453 ━━━━━━━━━━━━━━━━━━━━ 22s 49ms/step - accuracy: 0.9752 - loss: 0.0456 - val_accurac

c:\Users\Mostafa\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


453/453 ━━━━━━━━━━━━━━━━━━━━ 28s 49ms/step - accuracy: 0.5135 - loss: 0.3590 - val_accuracy: 0.8192 - val_loss: 0.2996
Epoch 2/40
453/453 ━━━━━━━━━━━━━━━━━━━━ 22s 48ms/step - accuracy: 0.8707 - loss: 0.1718 - val_accuracy: 0.9453 - val_loss: 0.1084
Epoch 3/40
453/453 ━━━━━━━━━━━━━━━━━━━━ 22s 48ms/step - accuracy: 0.9536 - loss: 0.0828 - val_accuracy: 0.9601 - val_loss: 0.0758
Epoch 4/40
453/453 ━━━━━━━━━━━━━━━━━━━━ 22s 48ms/step - accuracy: 0.9665 - loss: 0.0612 - val_accuracy: 0.9653 - val_loss: 0.0779
Epoch 5/40
453/453 ━━━━━━━━━━━━━━━━━━━━ 22s 48ms/step - accuracy: 0.9691 - loss: 0.0566 - val_accuracy: 0.9543 - val_loss: 0.0866
Epoch 6/40
453/453 ━━━━━━━━━━━━━━━━━━━━ 22s 48ms/step - accuracy: 0.9703 - loss: 0.0577 - val_accuracy: 0.9633 - val_loss: 0.0739
Epoch 7/40
453/453 ━━━━━━━━━━━━━━━━━━━━ 22s 48ms/step - accuracy: 0.9743 - loss: 0.0443 - val_accuracy: 0.9678 - val_loss: 0.0635
Epoch 8/40
453/453 ━━━━━━━━━━━━━━━━━━━━ 22s 48ms/step - accuracy: 0.9750 - loss: 0.0450 - val_accurac